In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import random
pyo.init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = 'notebook'
from matplotlib.animation import FuncAnimation


sys.path.append("/home/uttam/Dropbox/Projects/timeseries_generator/src/")
from generators import Particle

In [5]:
dimension = 2
particle = Particle(location=np.zeros(dimension), fertility=0.5, mortality=0.05, diffusion_quantity="velocity")
particles_dict = {}
particles_dict[0] = particle
particle_id_max = 0
locations_df = pd.DataFrame({"time": 0.0, "particle_id": [0], "x": [0.0], "y": [0.0]})
recording_interval = 10

time = 0.0
dtime = 1.0
for iterate in range(1000):
    new_particles_list = []
    killed_particles_list = []
    total_particles = len(particles_dict)
    for particle_id in particles_dict.keys():
        particle = particles_dict[particle_id]
        particle.move(dtime=dtime)
        reproduce = particle.reproduce(dtime=dtime, custom_parameters={"fertility": 0.5 * (1.0 - total_particles / 1000.0)})
        if reproduce:
            new_particles_list.append(reproduce)
        survive = particle.survive(dtime=dtime)
        if not survive:
            killed_particles_list.append(particle_id)
    if len(new_particles_list) > 0:
        for i, particle in enumerate(new_particles_list):
            particles_dict[particle_id_max + i + 1] = particle
        particle_id_max = particle_id_max + len(new_particles_list)
    if len(killed_particles_list) > 0:
        for particle_id in killed_particles_list:
            del particles_dict[particle_id]
    time += dtime
    if iterate % recording_interval == 0:
        particles_id_list = list(particles_dict.keys())
        x_list = [particles_dict[particle_id].location[0] for particle_id in particles_id_list]
        y_list = [particles_dict[particle_id].location[1] for particle_id in particles_id_list]
        locations_df = pd.concat([
            locations_df,
            pd.DataFrame({
                "time": time * np.ones(len(particles_dict)),
                "particle_id": particles_id_list,
                "x": x_list,
                "y": y_list
            })
        ])
    if len(particles_dict) == 0 or len(locations_df) > 500000:
        break
    if iterate % 10 == 0:
        print(iterate, end=", ")

0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 

In [6]:
locations_df["size"] = 20
len(locations_df)

93385

In [7]:
fig = px.scatter(
    locations_df,
    x="x", y="y", animation_frame="time", animation_group="particle_id",
    color="particle_id", hover_name="particle_id",
    range_x=[locations_df["x"].min(),locations_df["x"].max()], range_y=[locations_df["y"].min(), locations_df["y"].max()],
    size="size", 
)
fig.write_html("/home/uttam/Dropbox/Projects/timeseries_generator/figures/test.html")